In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from Wappalyzer import Wappalyzer, WebPage
import mysql.connector as my

In [30]:
data=pd.read_csv(r"C:/Users/innoppl/Sorted_durpal 10_ March 2024.csv")
Domains=data['Domain']
# Domains=Domains[:5000]
Domains

0                      un.org
1                     cdc.gov
2               worldbank.org
3                    usda.gov
4              greenpeace.org
                 ...         
28757    apostilleservice.org
28758             vorab.jetzt
28759                vordt.be
28760           app.wtccjc.tw
28761           topteplica.ru
Name: Domain, Length: 28762, dtype: object

In [31]:
con= my.connect(host='localhost',user='root',password='Jasper@1998', database= 'Techstack')
cursor=con.cursor()
cursor.execute('SHOW tables') #executing show database to check the databases present in MYSQL server
cursor.fetchall() 

[('automotive',), ('aws',), ('cms_tech',), ('magento_1',), ('shopify',)]

In [4]:
# table_query=''' CREATE TABLE cms_tech (
#                 URL varchar(255),
#                 CMS varchar(255))'''
# cursor.execute(table_query)
cursor.execute('SHOW tables') 
cursor.fetchall() 

[('automotive',), ('aws',), ('cms_tech',), ('magento_1',), ('shopify',)]

In [44]:
query ='''SELECT * FROM cms_tech
where list_info ='Drupal 10';'''

cursor.execute(query)
check=cursor.fetchall()
col= [colo[0] for colo in cursor.description]
cart_df= pd.DataFrame(check , columns=col)
cart_df

,URL,CMS,list_info,created_at
0,irs.gov,Drupal 10,Drupal 10,2024-03-22 17:16:01
1,usda.gov,Drupal 10,Drupal 10,2024-03-22 17:16:02
2,pega.com,Drupal 10,Drupal 10,2024-03-22 17:16:04
3,nist.gov,Drupal 10,Drupal 10,2024-03-22 17:16:04
4,denverlibrary.org,Drupal 10,Drupal 10,2024-03-22 17:16:05
...,...,...,...,...
31158,dionnagreen.com,Drupal 10,Drupal 10,2024-03-28 17:58:15
31159,civiclf.org,Drupal 10,Drupal 10,2024-03-28 17:58:15
31160,digitalstuff.dev,Drupal 9,Drupal 10,2024-03-28 17:58:15
31161,diogenesinabarrel.com,Drupal 10,Drupal 10,2024-03-28 17:58:15


30140

In [45]:
domains= data[~data['Domain'].isin(cart_df['URL'])]
Domains= domains['Domain']
Domains

9           treasury.gov
71           unwomen.org
84            gouv.qc.ca
137          state.tx.us
167            jst.go.jp
              ...       
28741        apps.rnp.br
28745    applemevpn.site
28751        app.clubmed
28758        vorab.jetzt
28759           vordt.be
Name: Domain, Length: 3105, dtype: object

In [41]:
insert_query = "INSERT INTO cms_tech (URL) VALUES (%s)"
update_query = "UPDATE cms_tech SET CMS = %s, list_info = %s  WHERE URL = %s"

for domain in Domains:
    list_info= 'Drupal 10'
    url = "https://builtwith.com/"+domain
    cursor.execute(insert_query, (domain,))
    con.commit()
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    div_tags = soup.find_all('div', class_='card mt-4 mb-2')
    for div_tag in div_tags:
        if "Content Management System" in div_tag.get_text():
        # Find all tags inside the div tag
#             tags = div_tag.find_all('div',class_='row mb-2 mt-2')#currently this class suspended
            tags = div_tag.find_all('div',class_='row mb-1 mt-1')
            try:
                last_tag_text = tags[-1].find()
                if last_tag_text.find_all('h5'):
                    cms= last_tag_text.find_all('h5')[-1].text
                    cursor.execute(update_query, (cms, list_info,domain))
                    con.commit()
                else:
                    cms= last_tag_text.find('h2').text
                    cursor.execute(update_query, (cms,list_info, domain))
                    con.commit()
            except:
                cursor.execute(update_query, ('Error',list_info, domain))
                con.commit()
                

ReadTimeout: HTTPSConnectionPool(host='builtwith.com', port=443): Read timed out. (read timeout=None)

In [47]:
cart_df.rename(columns={'URL':'Domain'}, inplace=True)

In [51]:
final= pd.merge(data, cart_df, on = 'Domain', how='left')

In [56]:
final.drop_duplicates(subset='Domain').to_csv('D:\Drupal10 overalldata.csv', index=False)